# Extending Pytorch
## 1. Extending torch.autograd
### 1.1 用法理解
1. 这里的扩展指的是以自定义autograd.Function的方式增加autograd处理的operation
2. **使用场景：**\
(1)想在模型中使用不可导函数\
(2)虽然依赖non-Pytorch library(比如用numpy)来implement operation，但仍然想让operation可以chain with其他pytorch library提供的operations，并且使用autograd engine来做forward和backward。\
(3)为了提升内存利用率或者performance，使用了c++ extension来写operation，也可以wrap成Function来应用autograd engine。\
(4)为了减少内存占用，想要减少number of buffers saved for backward pass，也可以用自动以函数来combine ops together。
3. **不要使用自定义Function的场景：**\
(1)pytorch函数库已经有想要执行的运算，而且已经可以record backward Graph，就没有必要自己写。 \
(2)如果只是想要maintain state，比如：trainable parameters，可以用自定义的module，而不需要自定义Function。 \
(3)如果想要改变pytorch库中函数在backward pass中计算gradient的方式，以实现其他的side effect，可以用registering a tensor或者registering a Module hook的方式实现，也没有必要额外定义函数。

### 1.2 4. 实现方式
**step1: 定义Function的子类，并在其中implement三个methods：forward(), setup_context(), backward()。**
1. forward()：\
(1)任意python的数据类型都可以作为forward的input。\
(2)如果用tensor作为argument，并且该tensor要track history(设置了requires_grad=True)，那么在调用forward之前，会被转换成不需要track history的tensor，他们在forward中的应用也会计入graph。如果用list or dict of tensor，那么上述规则不会自动发生。\
(3)可以返回single tensor作为单一output，或者a tuple of tensors作为multiple outputs。
2. setup_context()：只处理信息保存，不能做computation。
3. backward(),即vjp()：定义梯度计算公式。\
(1)它收到的inputs数量应该和函数outputs数量相同，对应的就是这些outputs各自的梯度。一定不能对这些inputs做in-place modification。\
(2)它返回的output tensors数量应该和函数的inputs数量相同。对应的就是各个inputs的梯度。<font color=blue>如果有的inputs不需要计算梯度，或者他们不是tensor类型</font>，那么也要用None作为它的梯度返回值。\
(3)如果forward()中有optional argument，那么backward要预留返回值的位置，此时返回的gradient数量可能超过实际input arguments数量，但只要将他们都取值为None就行。 

**step2: 合理使用ctx，确保新定义的函数可以使用autograd engine。** 
1. **save_for_backward()**: 存放backward中要使用的tensor，non-tensor直接存在ctx上作为attribute。<font color=red>如果既不是input，又不是output的tensor要在ctx中存放供backward使用，则整个函数可能无法支持double backward。</font>
2. <font color=green>**mark_dirty()**</font>: 如果有input被modified in-place，要用mark_dirty做标记
3. <font color=green>**mark_non_differetiable()**</font>: 如果有的output无法求梯度（不可微），要用该method来做标记。默认所有可微的outputs tensor都会被设置为requires_grad=True。一旦mark后，就不会被设置为requires_grad=True。
4. <font color=green>**set_materialize_grads()**</font>: 默认设置为True。当函数input的梯度与backward的arguments(dout)无关时，可以设参数为False优化梯度计算。用于告诉autograd engine not to materialize grad tensors given to backward function。也就是设置False后, backward的参数中的None object in Python or “undefined tensor” in C++将不会在调用backward之前被转变成<font colro=red>**全零tensor**</font>, 此时需要手动handle这些objects，as if they were tensors filled with zeros.

**step3: 如果函数不支持double backward，要对backward method用decorate once_differentiable()来明确声明。** \
**step4: 用torch.autograd.gradcheck()来检查backward的定义方式是否正确**

In [1]:
# 例1：自定义线性函数 y = x @ w.T + b
import torch
from torch.autograd import Function

class LinearFunction(Function):

    @staticmethod
    def forward(input, weight, bias):
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    @staticmethod
    # inputs是传给forward的所有inputs构成的tuple
    # output是forward()的output
    def setup_context(ctx, inputs, output):
        input, weight, bias = inputs
        ctx.save_for_backward(input, weight, bias)

    # 本例中的函数只有单一output, 所以backward只输出1个gradient
    @staticmethod
    def backward(ctx, grad_output):
        # unpack saved_tensors
        input, weight, bias = ctx.saved_tensors
        
        # inputs中所有元素的梯度都初始化为None，又additional trailing Nones会被忽略
        # 所以当函数有optional inputs的时候，return statement可以很简单
        grad_input = grad_weight = grad_bias = None

        # 这里的条件判断optional，可以提升效率
        if ctx.needs_input_grad[0]:   
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0)

        # 给不需要gradient的那些input也返回gradient并不会报错
        return grad_input, grad_weight, grad_bias

**使用自定义Function的两种常见方式：**
1. 直接apply成一个函数
2. wrap成一个新的函数：此时支持default args and keyword args

In [2]:
# Option 1: 直接apply一个函数
linear = LinearFunction.apply

# Option 2: wrap in a function
def linear(input, weight, bias=None):
    return LinearFunction.apply(input, weight, bias)

In [3]:
## 例2，使用non-tensor argument: y = const * x
class MulConstant(Function):
    @staticmethod
    def forward(tensor, constant):
        return tensor * constant

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx是一个context object，用于为backward存储所需信息
        tensor, constant = inputs
        ctx.constant = constant   # constant直接存为ctx的attribute

    @staticmethod
    def backward(ctx, grad_output):
        # 所有input都要有梯度返回值，non-Tensor arguments的梯度返回值为None
        return grad_output * ctx.constant, None

In [4]:
## 例3，对上例做优化：constant的梯度是None，与backward()的argument无关
class MulConstant(Function):
    @staticmethod
    def forward(tensor, constant):
        return tensor * constant

    @staticmethod
    def setup_context(ctx, inputs, output):
        tensor, constant = inputs
        # constant的梯度返回值是None，与backward的argument无关
        # 所以设置set_materialize_grads(False)
        ctx.set_materialize_grads(False)
        ctx.constant = constant

    @staticmethod
    def backward(ctx, grad_output):
        # 设置set_materialize_grads(False)后，
        # 要手动处理grad_output为None的情形，本例中直接return None就行
        if grad_output is None:
            return None, None

        return grad_output * ctx.constant, None

**如果需要保存forward()中计算出来的中间值tensor**
1. 有两种处理方式：\
(1)将其处理成forward的outputs。 <font color=blue>要计算高阶导数的时候，要用这种方式，同时还要将该tensor存到ctx.save_for_backward()中。</font> \
说明：\
backward的input，如grad_output, 也可能requires_grad=True。如果backward中用的operation是可微的，可以进一步计算高阶导数。但是ctx中存储的tensor本身并不会有gradients flowing back for them. 如果需要gradient flowing back到这些tensor上，就要将他们处理成output of the custom Function，同时存在ctx.save_for_backward()中. 这样，tensor就既能被backward()使用，又能有gradient flowing back to it。\
(2)联用forward和setup_context
2. 如果计算图要通过该tensor，就要为他定义gradient的计算公式。 

In [5]:
## 处理成forward的outputs
# 例4：支持高阶导数
class MyCube(torch.autograd.Function):
    @staticmethod
    def forward(x):
        # 要存dx在backward中使用，为此，在forward中将其作为返回值
        dx = 3 * x ** 2
        result = x ** 3
        return result, dx

    @staticmethod
    def setup_context(ctx, inputs, output):
        x, = inputs
        result, dx = output
        ctx.save_for_backward(x, dx)

    @staticmethod
    def backward(ctx, grad_output, grad_dx):
        x, dx = ctx.saved_tensors
        # In order for the autograd.Function to work with higher-order
        # gradients, we must add the gradient contribution of `dx`,
        # which is grad_dx * 6 * x.
        result = grad_output * dx + grad_dx * 6 * x
        return result

# Wrap MyCube in a function以便确定唯一的output值
def my_cube(x):
    result, dx = MyCube.apply(x)
    return result

In [6]:
# 例5：check gradient
from torch.autograd import gradcheck

# gradcheck的输入要处理成a tuple of tensors
input = (torch.randn(20,20,dtype=torch.double,requires_grad=True), torch.randn(30,20,dtype=torch.double,requires_grad=True))
test = gradcheck(linear, input, eps=1e-6, atol=1e-4)
print(test)

True


In [7]:
# 例6：将forward和setup_context合并到forward中（not recommended）
class LinearFunction(Function):
    @staticmethod
    # ctx is the first argument to forward
    def forward(ctx, input, weight, bias=None):
        # The forward pass can use ctx.
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0)

        return grad_input, grad_weight, grad_bias

## 2. Extending torch.nn
nn输出两种类型的interface：modules和他们的functional version。\
extend nn可以用上述两种方式，但建议：\
(1)如果layer中有parameters或者buffers，用module。如：conv, affine. \
(2)如果没有，用function，比如activation function, pooling, relu.

In [8]:
import torch.nn as nn

class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super().__init__()
        self.input_features = input_features
        self.output_features = output_features

        # 1. nn.Parameter is a special kind of Tensor, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. 
        # 2. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # 3. nn.Parameters require gradients by default.
        self.weight = nn.Parameter(torch.empty(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.empty(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        nn.init.uniform_(self.weight, -0.1, 0.1)
        if self.bias is not None:
            nn.init.uniform_(self.bias, -0.1, 0.1)

    def forward(self, input):
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. 
        return 'input_features={}, output_features={}, bias={}'.format(
            self.input_features, self.output_features, self.bias is not None
        )

## 3. Extending torch
1. 目标：创建与tensor类似的自定义python type，既可以是与tensor所实现的操作相似但不相同的数据类型，也可以是tensor的子类型。在自定义的class中定义相应的同名methods，让pytorch中torch namespace里面原本接受tensor operands(操作数)的函数也能处理该自定义的数据类型。
2. pytorch中提供的机制：如果自定义的python type class中定义了名为\_\_torch_function\_\_()的method，那么当自定义类型的实例作为参数传给torch namespace的函数的时候，pytorch就会invoke这个\_\_torch_function\_\_()。\
通过这种方式，可以自定义torch namespace中各种函数的implementation。当以自定义类型为参数来调用torch中的函数时，\_\_torch_function\_\_()实际上会调用这里自定义的implementation。

### 3.1 自定义与tensor类似的数据类型

In [9]:
## 例1：自定义一个2D scalar tensor
#  1. n*n matrix，每个元素都是一个tensor scalar
#  2. 对角线上的n个元素的值相同，由arguments决定，其他都是0

class ScalarTensor(object):
    def __init__(self, N, value):
        self._N = N
        self._value = value
    def __repr__(self):
        return f"ScalarTensor(N={self._N}, value={self._value})"
    def tensor(self):
        return self._value * torch.eye(self._N)

In [10]:
d = ScalarTensor(3, 2)
print(d)
print(d.tensor())

ScalarTensor(N=3, value=2)
tensor([[2., 0., 0.],
        [0., 2., 0.],
        [0., 0., 2.]])


In [11]:
#  此时没有定义__torch_function__(),不支持torch operation
torch.mean(d)

TypeError: mean(): argument 'input' (position 1) must be Tensor, not ScalarTensor

### 让自定义类型支持自定义torch operation
1. 定义__torch_function__()，按照规则定义好4个参数 \
(1)func: reference to the torch API function that is being overridden \
(2)types: 说明__torch_function__ 支持的type list\
(3)args: 传递给function的arguments tuple \
(4)kwargs: 传递给function的keyword arguments dict
2. 定义该类型数据上与torch operation匹配的运算

In [12]:
## 例2：让自定义类型支持自定义torch operation: torch.mean()

# 1.指定该数据类型implement的operation，函数名要与torch namespace中的匹配
HANDLED_FUNCTIONS = {}

class ScalarTensor(object):
    def __init__(self, N, value):
        self._N = N
        self._value = value

    def __repr__(self):
        return "ScalarTensor(N={}, value={})".format(self._N, self._value)

    def tensor(self):
        return self._value * torch.eye(self._N)

    @classmethod
    def __torch_function__(cls, func, types, args=(), kwargs=None):
        if kwargs is None:
            kwargs = {}
        if func not in HANDLED_FUNCTIONS or not all(
            issubclass(t, (torch.Tensor, ScalarTensor))
            for t in types
        ):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)

In [13]:
# 2.定义operation的implementation，这里是torch.mean
import functools

# 定义一个decorator
def implements(torch_function):
    """Register a torch function override for ScalarTensor"""
    def decorator(func):
        functools.update_wrapper(func, torch_function)
        HANDLED_FUNCTIONS[torch_function] = func
        return func
    return decorator

# 用decorator来实现具体函数的implementation
@implements(torch.mean)
def mean(input):
    return float(input._value) / input._N

In [14]:
d = ScalarTensor(5, 2)
torch.mean(d)

0.4

In [15]:
# 3. 函数operand有多个，其中有的是tensor，有的是自定义的类型
def ensure_tensor(data):
    # torch.tensor()通过深拷贝数据，构造一个新tensor
    if isinstance(data, ScalarTensor):
        return data.tensor()
    # as_tensor可以将python对象转换为tensor
    return torch.as_tensor(data)

@implements(torch.add)
def add(input, other):
    # 如果输入都是ScalarTensor，加总两个底层tensor
    try:
        if input._N == other._N:
            return ScalarTensor(input._N, input._value + other._value)
        else:
            raise ValueError("Shape mismatch!")
    # 如果两种输入中有ScalarTensor又有Tensor，要将scalarTensor转变成tensor
    except AttributeError:
        return torch.add(ensure_tensor(input), ensure_tensor(other))

In [16]:
s = ScalarTensor(2, 2)
print(torch.add(s, s))

t = torch.tensor([[1, 1,], [1, 1]])
print(torch.add(s, t))

ScalarTensor(N=2, value=4)
tensor([[3., 1.],
        [1., 3.]])


### 处理torch namespace中有，但是没有自定义函数implementation的情形
1. 如果对自定义类型用torch namespace中的函数，但没有implement对应函数，会报错
2. 一种不报错的处理方式是，判断自定义类型中有tensor method，调用该method把tensor传给torch原函数处理。<font color=red>注意这是因为自定义类型中一般都有tensor method。此时也要判断原生的运算规则是否符合自定义类型本身的需求。</font>

In [17]:
torch.mul(s, 3)

TypeError: no implementation found for 'torch.mul' on types that implement __torch_function__: [<class '__main__.ScalarTensor'>]

In [22]:
# invoke tensor method生成tensor对象后，调用torch原有的函数
class ScalarTensor(object):
    def __init__(self, N, value):
        self._N = N
        self._value = value

    def __repr__(self):
        return "ScalarTensor(N={}, value={})".format(self._N, self._value)

    def tensor(self):
        return self._value * torch.eye(self._N)
    
    # 改变__torch_function__中的执行逻辑
    @classmethod
    def __torch_function__(cls, func, types, args=(), kwargs=None):
        if kwargs is None:
            kwargs = {}
        if func not in HANDLED_FUNCTIONS or not all(
                issubclass(t, (torch.Tensor, ScalarTensor))
                for t in types
            ):
            args = [a.tensor() if hasattr(a, 'tensor') else a for a in args]
            return func(*args, **kwargs)
        return HANDLED_FUNCTIONS[func](*args, **kwargs)

In [23]:
s = ScalarTensor(2, 2)
torch.mul(s, s)

tensor([[4., 0.],
        [0., 4.]])

### 3.2 自定义tensor的子类型

### 3.3 自定义tensor wrapper类型